In [30]:
import advent
from advent.intcode import run
code = advent.get_intcode(23)


In [31]:
# The approach: We run each program for n (e.g. 100) steps
# we use 50 IO objects, but one IO manager that
# picks up the output after each run

In [32]:
class IO():
    def __init__(self, input_=[]):
        self.input_buffer = input_.copy()
        self.output_buffer = []
    
    def add_input(self, value, map_fn=int):
        for c in value:
            self.input_buffer.append(map_fn(c))
        return self

    def read(self):
        if len(self.input_buffer) == 0:
            return -1
        return self.input_buffer.pop(0)
    
    def write(self, value):
        self.output_buffer.append(value)
        return self
    
    def output(self):
        return self.output_buffer
    
    def remove_output(self, n=3):
        # remove the first n outputs
        self.output_buffer = self.output_buffer[n:]
        return self

In [33]:
def handle_output(ios, i):
    # The output looks like: ADDR, X, Y
    # if we have less than 3 outputs, just ignore it
    # if we have e.g. 6 or more outputs, handle all of them
    # if ADDR is 255 in part 1, raise an exception (easiest way to just stop)
    io = ios[i]
    while len(io.output()) >= 3:
        message = io.output()[:3]
        if message[0] == 255:
            raise RuntimeError(f"IO {i} returned y: {message[2]}")
        ios[message[0]].add_input([message[1], message[2]])
        io.remove_output()

In [34]:
ios = [IO([i]) for i in range(50)]
# each program has separate copy of the code that may be modified while running
codes = [code.copy() for _ in range(50)]
pointers = [0 for _ in range(50)]
relative = [0 for _ in range(50)]

while True:
    for i in range(50):
        state, _ = run(codes[i], ios[i], pointers[i], relative[i], 100) # run 100 steps
        pointers[i] = state[1]
        relative[i] = state[2]
        handle_output(ios, i)

RuntimeError: IO 45 returned y: 17949

In [37]:
# Part 2

# in part 2, we have to change handle_output: messages to addr 255 go to nat instead
# here, 'nat' is simply a tuple of (x, y)

def handle_output_part2(ios, i, nat):
    io = ios[i]
    while len(io.output()) >= 3:
        message = io.output()[:3]
        if message[0] == 255:
            nat = (message[1], message[2])
        else:
            ios[message[0]].add_input([message[1], message[2]])
        io.remove_output()
    return nat

def network_idle(ios):
    # This just checks if input_buffer is empty and output buffer are all len < 3
    # this doesn't *really* mean the network is idle, could just mean a message will be sent soon
    # so to be safe, run for a good amount of steps so any incoming output will be coming
    for io in ios:
        if len(io.input_buffer) > 0 or len(io.output()) >= 3:
            return False
    return True

In [40]:
ios = [IO([i]) for i in range(50)]
# each program has separate copy of the code that may be modified while running
codes = [code.copy() for _ in range(50)]
pointers = [0 for _ in range(50)]
relative = [0 for _ in range(50)]
last_y_value_to_0 = None
nat = (None, None)

while True:
    for i in range(50):
        # 100 is not enough this time to truly say the network is 'idle'
        state, _ = run(codes[i], ios[i], pointers[i], relative[i], 1000)
        pointers[i] = state[1]
        relative[i] = state[2]
        nat = handle_output_part2(ios, i, nat)
    if network_idle(ios):
        if nat[1] == last_y_value_to_0:
            raise RuntimeError(f"Value {nat[1]} was sent to 0 twice in a row!")
        ios[0].add_input([nat[0], nat[1]])
        last_y_value_to_0 = nat[1]

RuntimeError: Value 12326 was sent to 0 twice in a row!